#### PyTorch基础：审计网络包`nn`和优化器`optm`

`torch.nn`是专门为神经网络设计的模块化接口。`nn`构建于`Autograd`之上，可用来定义和运行神经网络。这里我们主要介绍的是几个一些常用的类。

In [1]:
import torch
import torch.nn as nn
torch.__version__

'2.8.0'

除了`nn`别名以外，我们还引用了`nn.functional`，这个包中包含了神经网络中使用的一些常用函数，这些函数的特点是，不具有可学习的参数(如`ReLU`，`pool`，`DropOut`等)，这些函数可以放在构造函数中，也可以不放。这里建议不放。

一般情况下我们会将`nn.functional`设置为大写的F，这样缩写方便调用。

In [2]:
import torch.nn.functional as F

#### 定义一个网络

PyTorch中已经为我们准备好了线程的网络模型，只要继承`nn.Module`，并实现它的`forward`方法，PyTorch会根据autograd，自动实现`backward`函数，在`forward`函数中可使用任何tensor支持的函数，还可以使用`if`,`for`循环，`print`,`log`等Python语法，写法和标准的Python写法一致。

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3) # 输入通道1，输出通道6，卷积核3x3
        self.fc1 = nn.Linear(1350, 10) # 全连接层，输入1350，输出10

    def forward(self, x):
        print(x.size())
        # 卷积 -> 激活 -> 池化 
        x = self.conv1(x)
        x = F.relu(x)
        print(x.size())

        x = F.max_pool2d(x, (2, 2))
        x = F.relu(x)

        print(x.size())

        x = x.view(x.size()[0], -1)
        print(x.size())
        x = self.fc1(x)
        return x

net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


网络的可学习通过`net.parameters()`返回

In [8]:
for parameters in net.parameters():
    print(parameters)

Parameter containing:
tensor([[[[ 0.2173,  0.0440,  0.2857],
          [ 0.2057, -0.0692,  0.2753],
          [-0.1319, -0.0223, -0.0573]]],


        [[[-0.2166, -0.2418, -0.2306],
          [ 0.2657,  0.0525, -0.1871],
          [ 0.0207,  0.1095, -0.1228]]],


        [[[ 0.1967,  0.1939,  0.3002],
          [-0.2791,  0.2969,  0.2089],
          [ 0.0929, -0.0466,  0.1772]]],


        [[[ 0.1503,  0.0284,  0.0694],
          [ 0.1241, -0.0491,  0.2438],
          [ 0.0805,  0.0167,  0.1870]]],


        [[[ 0.3273,  0.0702,  0.2784],
          [ 0.3303,  0.0772,  0.0736],
          [-0.2278,  0.2703, -0.0339]]],


        [[[ 0.2486,  0.1426, -0.1970],
          [-0.1535, -0.0873,  0.0620],
          [-0.1726, -0.2448,  0.3244]]]], requires_grad=True)
Parameter containing:
tensor([0.3050, 0.2006, 0.1481, 0.1062, 0.1506, 0.2509], requires_grad=True)
Parameter containing:
tensor([[ 0.0060, -0.0162, -0.0256,  ..., -0.0252, -0.0137,  0.0007],
        [ 0.0196,  0.0244, -0.0196,  ..., 

In [9]:
for name, parameters in net.named_parameters():
    print(name, ':', parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


In [10]:
input = torch.randn(1, 1, 32, 32) # batch_size=1, channel=1, height=32, width=32
out = net(input)  
out.size()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


torch.Size([1, 10])

In [11]:
input.size()

torch.Size([1, 1, 32, 32])

在反向传播前，先要将所有的参数梯度清零

In [12]:
net.zero_grad() 
out.backward(torch.randn(1,10))  #反向传播的实现是PyTorch自动实现的，我们只需要调用这个函数即可

**注意**: `torch.nn`只支持mini-batches，不支持一次只输入一个样本，即以此必须是一个batch。

也就是说，就算我们输入一个样本，也会对样本进行分批，所以，所有的输入都会增加一个维度，我们对比下刚才的input，`nn`中定义为3维，但是我们人工创建时多增加了一个维度，变为了4维，最前面的1即为batch-size。

#### 损失函数
在`nn`中PyTorch还预制了常用的损失函数，下面我们用MSELoss用来计算均方误差。

In [13]:
y = torch.arange(0, 10).view(1, 10).float() 
criterion = nn.MSELoss()
loss = criterion(out, y)

# loss是个scalar，我们可以直接用item获取到他的python类型的数值
print(loss.item())

27.068767547607422


#### 优化器

在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法（SGD）的更新策略如下：

`weight = weight - learning_rate * gradient`

在`torch.optim`中实现大多数的优化方法，例如`RMSProp`,`Adam`,`SGD`等，下面我们使用SGD做个简单的样例

In [15]:
import torch.optim

out = net(input)
criterion = nn.MSELoss()
loss = criterion(out, y)

# 新建一个优化器，SGD只需要调整的参数和学习率
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)

# 先清零梯度（与net.zero_grad()效果一样）
optimizer.zero_grad()
loss.backward()

optimizer.step()  # 更新所有参数

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


这样，神经网络的数据的一个完整的传播就已经通过PyTorch实现了，下面一章将介绍PyTorch提供的数据加载和处理工具，使用这些工具可以方便地处理所需要地数据。

#### PyTorch基础：数据的加载和预处理

PyTorch通过`torch.utils.data`对一般常用的数据加载进行了封装，可以很容易地实现多线程数据预读和批量加载。并且`torchvision`已经预先实现了常用图像数据集，包括前面使用过的`CIFAR-10`,`ImageNet`,`COCO`,`MINIST`,`LSUN`等数据集，可以通过`torchvision.datasets`方便调用。

In [16]:
import torch
torch.__version__

'2.8.0'

#### Dataset

`Dataset`是一个抽象类，为了能够方便地读取，需要将使用的数据包装为Dataset类。自定义的Dataset需要继承它并且实现两个成员方法：

1. `__getitem__()`该方法定义用索引（`0`到`len(self)`）获取一条数据或一个样本
2. `__len__()`该方法返回数据集的总长度

下面我们使用kaggle上的一个竞赛bluebook for bulldozers自定义一个数据集，为了方便介绍，我们使用里面的数据字典来做说明（因为条数少）


In [17]:
from torch.utils.data import Dataset
import pandas as pd

In [18]:
class BulldozerDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        return self.df.iloc[idx].SalePrice

至此，我们的数据集已经定义完成了，我们可以实例化一个对象访问它

In [20]:
ds_demo = BulldozerDataset('data/median_benchmark.csv') # 修改为实际路径

In [21]:
len(ds_demo)  # #实现了 __len__ 方法所以可以直接使用len获取数据总数

11573

In [22]:
ds_demo[0]  #用索引可以直接访问对应的数据，对应 __getitem__ 方法

np.float64(24000.0)

自定义的数据集已经创建好了，下面我们使用官方提供的数据载入器读取数据

#### Dataloader

DataLoader为我们提供了对Dataset的读取操作，常用参数有：`batch_size`（每个batch的大小），`shuffle`（是否进行shuffle操作），`num_workers`（加载数据的时候使用几个子进程）。下面做一个简单的操作

In [23]:
dl = torch.utils.data.DataLoader(ds_demo, batch_size = 4, shuffle = True, num_workers = 0)

DataLoader返回的是一个可迭代对象，我们可以使用迭代器分次获取数据

In [24]:
idata = iter(dl)
print(next(idata))

tensor([24000., 24000., 24000., 24000.], dtype=torch.float64)


常见的用法是使用for循环对其进行遍历：

In [25]:
for i, data in enumerate(dl):
    print(i, data)
    break

0 tensor([24000., 24000., 24000., 24000.], dtype=torch.float64)


我们已经可以通过`dataset`定义数据集，并使用`Dataloder`载入和遍历数据集，除了这些以外，PyTorch还能提供能`torcvison`的计算机视觉扩展包，里边封装了

- `torchvision`包
- `torchvision.datasets`：`torchvision.datasets`可以理解为PyTorch团队自定义的dataset，这些dataset帮我们提前处理好了很多的图片数据集，我们拿来就可以直接使用。

In [28]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))

])
trainset = datasets.MNIST(root = './data', 
                           train = True,  # 表示是否加载数据库的训练集，false的时候加载测试集
                           download = True,  #是否从网上下载数据集
                           transform = None) # 数据预处理操作

100.0%
100.0%
100.0%
100.0%


- `torchvision.models`不仅提供了常用图片数据集，还提供了训练好的模型，可以加载之后直接使用。

In [29]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained = True) #加载预训练的ResNet18模型

/Users/momokoyang/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/momokoyang/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
0.3%

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/momokoyang/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


- `torchvision.transforms`模块提供了一般的图像转换操作类，用作数据处理和数据增强

In [30]:
from torchvision import transforms as transforms
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #随机裁剪32x32的图片，边界填充4个像素
    transforms.RandomHorizontalFlip(),  #随机水平翻转图片
    transforms.RandomRotation((-45, 45)),  #随机旋转图片，旋转角度在-45到45度之间
    transforms.ToTensor(),  #将图片转换为Tensor格式
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  #R,G,B每层的归一化用到的均值和方差
])

(0.485, 0.456, 0.406), (0.2023, 0.1994, 0.2010) 这几个数字是什么意思？

官方的这个帖子有详细的说明: https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/21 这些都是根据ImageNet训练的归一化参数，可以直接使用，我们认为这个是固定值就可以